<div class='bar_title'></div>

*Data Driven Decision in Practice (D3IP): Urban Analytics*


# 3. Machine Learning with Scikit-learn

Gunther Gust <br>
Chair for Enterprise AI <br>
Data Driven Decisions (D3) Group <br>
Center for Artitifical Intelligence & Data Science (CAIDAS)



<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Content

## Building a machine learning pipeline

* Data Selection
* Model Training
* Model Evaluation
* Hyperparameter Tuning
* Data Cleaning

Credits: Most of the material of this lecture is adopted from www.kaggle.com

## Scenario: Real estate price predictions


This lecture provides an overview of how machine learning models can be used for real problems. We will build models as well as a machine learning pipeline based on the following scenario:

Your cousin has made millions of dollars speculating on real estate. He's offered to become business partners with you because of your interest in data science. He'll supply the money, and you'll supply models that predict how much various houses are worth.

You ask your cousin how he's predicted real estate values in the past and he says it is just intuition. But more questioning reveals that he's identified price patterns from houses he has seen in the past, and he uses those patterns to make predictions for new houses he is considering.

Instead of using intuition to make good decision, we want to train a __machine learning model to predict the value of new houses__.

## Loading the Data
The first step in any machine learning project is to load and familiarize yourself with the data. To this end, we can use the pandas library from last week and load the dataset with the following commands:

In [1]:
import pandas as pd

In [5]:
melbourne_file_path = 'https://raw.githubusercontent.com/GuntherGust/D3IP_data/main/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## Removing Missing Values

For simplicity we remove rows with missing values for this example. Note that a missing value can sometimes be a valuable information. 

In [6]:
print(melbourne_data.shape)
melbourne_data.dropna(axis=0, inplace=True)
print(melbourne_data.shape)

(13580, 21)
(6196, 21)


## Select data for modeling
On a first glimpse, we see that our dataset has too many variables to wrap our heads around. How can we pare down this overwhelming amount of data to something we can understand?

We'll start by picking a few variables using our intuition. To choose variables, we'll need to see a list of all columns in the dataset. That is done with the columns property of the DataFrame:

In [7]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

### Selecting the prediction target

To train a predictive model using supervised machine learning techniques, we have to identify the target variable. In the problem at hand, we want to predict the house prices. This information is encoded in the column Price.

By convention, the target variable is called **y**. 

In [8]:
y = melbourne_data['Price']

### Choosing "Features"
The columns that serve as input for our model (and are later used to make predictions) are called "features." Sometimes, you will use all columns except the target as features. Other times you'll be better off with fewer features.

For now, we'll build a model with only a few features. 

We select multiple features by providing a list of column names inside brackets. Each item in that list should be a string (with quotes).


In [9]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']

By convention, this data is called **X**.

In [10]:
X = melbourne_data[melbourne_features]

## Quick Exploratory Data Analysis (EDA)

Let's quickly review the data we'll be using to predict house prices using the describe method and the head method, which shows the top few rows. Visually checking your data with these commands is an important part of a data scientist's job. You'll frequently find surprises in the dataset that deserve further inspection.

In [11]:
print(X.shape)
X.head()

(6196, 7)


,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
1,2,1.0,156.0,79.0,1900.0,-37.8079,144.9934
2,3,2.0,134.0,150.0,1900.0,-37.8093,144.9944
4,4,1.0,120.0,142.0,2014.0,-37.8072,144.9941
6,3,2.0,245.0,210.0,1910.0,-37.8024,144.9993
7,2,1.0,256.0,107.0,1890.0,-37.8060,144.9954


In [12]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,6196.0,2.931407,0.971079,1.00000,2.000000,3.00000,4.0000,8.00000
Bathroom,6196.0,1.576340,0.711362,1.00000,1.000000,1.00000,2.0000,8.00000
Landsize,6196.0,471.006940,897.449881,0.00000,152.000000,373.00000,628.0000,37000.00000
BuildingArea,6196.0,141.568645,90.834824,0.00000,91.000000,124.00000,170.0000,3112.00000
YearBuilt,6196.0,1964.081988,38.105673,1196.00000,1940.000000,1970.00000,2000.0000,2018.00000
Lattitude,6196.0,-37.807904,0.075850,-38.16492,-37.855438,-37.80225,-37.7582,-37.45709
Longtitude,6196.0,144.990201,0.099165,144.54237,144.926198,144.99580,145.0527,145.52635


## Building Models in Scikit-learn


For now, we will use the scikit-learn library to create our models. As you will see in the upcoming section, this library is written as sklearn in the code. Scikit-learn offers a lot of powerful features and is easily the most popular library for modeling tabular data.

The steps to building and using a model in Scikit-learn are:
* __Define__:  What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.
* __Fit__: Capture patterns from our input data.
* __Predict__: Make predictions using input variables and the trained model.
* __Evaluate__: Determine how accurate the model's predictions are.

### Our first Decision Tree


Example decision tree: 

<div align="center">
  <img src="images/sample_data_decision_tree.png"  alt="First Image" style="width: 45%; display: inline-block; margin-right: -4px;" />
  <img src="images/decision_tree.png" alt="Second Image" style="width: 45%; display: inline-block; margin-left: -4px;" />
</div>


For a recap on decision trees, see e.g. [this post](https://www.analyticsvidhya.com/blog/2021/08/decision-tree-algorithm/). For a rigorous formal treatment, see e.g. [The Elements of Statistical Learning by Hastie et al (2008)](https://hastie.su.domains/ElemStatLearn/).

Here is a simple example of defining a decision tree model with scikit-learn and fitting it with the features and target variable selected above:

In [13]:
from sklearn.tree import DecisionTreeRegressor

# Define
melbourne_model = DecisionTreeRegressor(random_state=1)

# Fit
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

Many machine learning models allow some randomness in model training. Specifying a number for random_state ensures you get the same results in each run. This is considered a good practice. You use any number, and model quality won't depend meaningfully on exactly what value you choose.

We now have a fitted model that we can use to make predictions.

In practice, you'll want to make predictions for new houses coming on the market rather than the houses we already have prices for. But we'll make predictions for the first few rows of the training data to see how the predict function works.

In [14]:
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(melbourne_model.predict(X.head()))

Making predictions for the following 5 houses:
   Rooms  Bathroom  Landsize  BuildingArea  YearBuilt  Lattitude  Longtitude
1      2       1.0     156.0          79.0     1900.0   -37.8079    144.9934
2      3       2.0     134.0         150.0     1900.0   -37.8093    144.9944
4      4       1.0     120.0         142.0     2014.0   -37.8072    144.9941
6      3       2.0     245.0         210.0     1910.0   -37.8024    144.9993
7      2       1.0     256.0         107.0     1890.0   -37.8060    144.9954
The predictions are
[1035000. 1465000. 1600000. 1876000. 1636000.]


## Model Validation



We have successfully trained our very first model. However, we have no clue how good our model is. Yet, measuring model quality is the key to iteratively improving our models.

In most (though not all) applications, the relevant measure of model quality is predictive accuracy. In other words, will the model's predictions be close to what actually happens.

### Metrics



To evaluate the performance of our model we need to find a way to summarize the model quality in an understandable way. If we compare predicted and actual home values in our example dataset for 10,000 houses, we will find a mix of good and bad predictions. However, looking through a list of 10,000 predicted and actual values would be pointless. We need to summarize this into a single metric.

``Mean absolute error (MAE) = Mean (|actual price − predicted price|)``

There are many metrics for summarizing model quality, but we'll start with one called Mean Absolute Error (**MAE**). Let's break down this metric starting with the last word, error.

The prediction error for each house is: 

``error=actual−predicted``

So, if a house cost 150,000 Euros and you predicted it would cost 100,000 Dollars the error is 50,000 Dollars.

With the MAE metric, we take the absolute value of each error. This converts each error to a positive number. We then take the average of those absolute errors. This is our measure of model quality.

We could implement a function to calculate this metric (or any other metric) on our dataframe. However, Scikit-learn provides implementations of the most common metrics that can be easily imported.

In [15]:
from sklearn.metrics import mean_absolute_error

So lets use our decision tree model to make predictions for all observations in our dataset and calculate the MAE:

In [16]:
predicted_home_prices = melbourne_model.predict(X)
mae = mean_absolute_error(y, predicted_home_prices)

print("The MAE of our model is: {}".format(mae))

The MAE of our model is: 434.71594577146544


## In-Sample vs. Out-of-Sample Scores

The MAE of our model looks __very promising__. However, we used a single "sample" of houses for both building the model and evaluating it. Hence, the measure we just computed can be called an __"in-sample" score__.

Trusting the in-sample score to evaluate a model is __very dangerous__. Imagine that there is a variable in the dataset that is unrelated to the home price (e.g., the name of the current owner). However, in the sample of data we used to build the model, all names are unique and hence, all house prices in the sample can be explained by this feature. Our model will see this pattern and it will try to apply it to new datasets. 

Since models' practical value come from making predictions on __new data__, we measure performance on data that wasn't used to build the model. The most straightforward way to do this is to exclude some data from the model-building process, and then use those to test the model's accuracy on data it hasn't seen before. This data is called __validation data__.

The scikit-learn library has a function train_test_split to break up the data into two pieces. We'll use some of that data as training data to fit the model, and we'll use the other data as validation data to calculate the MAE.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
# Split the data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# Build the model
melbourne_model = DecisionTreeRegressor(random_state=1)
melbourne_model.fit(train_X, train_y)

# Evaluate the performance
val_predictions = melbourne_model.predict(val_X)
val_mae = mean_absolute_error(val_y, val_predictions)

print("The MAE of our model is: {}".format(val_mae))

The MAE of our model is: 262494.3027759845


The MAE for the in-sample data was about __500 Dollars__. Out-of-sample it is __more than 250,000 Dollars__.

This is the difference between a model that is almost exactly right, and one that is unusable for most practical purposes. As a point of reference, the average home value in the validation data is about __1.1 million dollars__. So the error in new data is about __a quarter__ of the average home value.

There are many ways to __tune and improve this model__, such as experimenting to find better features or different model types.

## Model Tuning

### Example: Varying the Depth of the Decision Tree



Now that we have a reliable way to measure the model performance, we can experiment with different parameters of the decision tree (or entirely different models) and see which combination gives us the best predictions. You can find the available models and parameters in the Scikit-learn [documentation](https://scikit-learn.org/stable/modules/classes.html).

In this simple example we will stick with our decision tree model and only vary one of the parameters.

You can see in the decision tree [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) that the model has many parameters (more than you'll want or need for a long time). The most important parameters determine the tree's depth. 

In practice, it's not uncommon for a tree to have 10 splits between the top level (all houses) and a leaf. As the tree gets deeper, the dataset gets sliced up into leaves with fewer houses. If a tree only had 1 split, it divides the data into 2 groups. If each group is split again, we would get 4 groups of houses. Splitting each of those again would create 8 groups. If we keep doubling the number of groups by adding more splits at each level, we'll have $2^{10}$  groups of houses by the time we get to the 10th level. That's 1024 leaves.

When we divide the houses amongst many leaves, we also have fewer houses in each leaf. Leaves with very few houses will make predictions that are quite close to those homes' actual values, but they may make very unreliable predictions for new data (because each prediction is based on only a few houses). This is a phenomenon called **overfitting**, where a model matches the training data almost perfectly, but does poorly in validation and other new data. 

On the flip side, if we make our tree very shallow, it doesn't divide up the houses into very distinct groups. At an extreme, if a tree divides houses into only 2 or 4, each group still has a wide variety of houses. Resulting predictions may be far off for most houses, even in the training data (and it will be bad in validation too for the same reason). When a model fails to capture important distinctions and patterns in the data, so it performs poorly even in training data, that is called **underfitting**.

Since we care about accuracy on new data, which we estimate from our validation data, we want to find the sweet spot between underfitting and overfitting.

There are a few alternatives for controlling the tree depth, and many allow for some routes through the tree to have greater depth than other routes. But the max_leaf_nodes argument provides a very sensible way to control overfitting vs underfitting. The more leaves we allow the model to make, the more we move from the underfitting area in the above graph to the overfitting area.

We write a short utility function to help compare MAE scores from different values for max_leaf_nodes:

In [16]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

Next, we loop over different values for the parameter to compare the in-sample and the out-of-sample performance of our model:

In [17]:
for max_leaf_nodes in [2, 5, 50, 500, 5000, 10000]:
    is_mae = get_mae(max_leaf_nodes, X, X, y, y)
    oos_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t In-sample:  %d \t Out-of-sample:  %d" %(max_leaf_nodes, is_mae, oos_mae))

Max leaf nodes: 2  	 In-sample:  411930 	 Out-of-sample:  433608
Max leaf nodes: 5  	 In-sample:  325389 	 Out-of-sample:  347380
Max leaf nodes: 50  	 In-sample:  221593 	 Out-of-sample:  258171
Max leaf nodes: 500  	 In-sample:  116715 	 Out-of-sample:  246793
Max leaf nodes: 5000  	 In-sample:  1881 	 Out-of-sample:  261451
Max leaf nodes: 10000  	 In-sample:  434 	 Out-of-sample:  261451


Here's the takeaway: Models can suffer from either:

- Overfitting: capturing spurious patterns that won't recur in the future, leading to less accurate predictions, or
- Underfitting: failing to capture relevant patterns, again leading to less accurate predictions.

We use validation data, which isn't used in model training, to measure a candidate model's accuracy. This lets us try many candidate models and keep the best one.

## Training a Random Forest



Decision trees leave us with a difficult trade-off. A deep tree with lots of leaves will overfit because each prediction is coming from historical data from only the few houses at its leaf. But a shallow tree with few leaves will perform poorly because it fails to capture as many distinctions in the raw data.

Even today's most sophisticated modeling techniques face this tension between underfitting and overfitting. But, many models have clever ideas that can lead to better performance. We'll look at the random forest as an example.

The random forest uses __many trees__, and it makes a prediction by __averaging the predictions of each component tree__. It generally has much better predictive accuracy than a single decision tree and it works well with default parameters. If you keep modeling, you can learn more models with even better performance, but many of those are sensitive to getting the right parameters.

Thanks to our Scikit-learn modeling pipeline we can reuse most of our code to train a random forest model with 100 trees.

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
# Define
forest_model = RandomForestRegressor(random_state=1, n_estimators=100)

# Fit
forest_model.fit(train_X, train_y)

# Evaluate
melb_preds = forest_model.predict(val_X) 
print("The MAE of our model is: {}".format(mean_absolute_error(val_y, melb_preds)))

The MAE of our model is: 191669.7536453626


There is likely room for further improvement, but this is a __big improvement over the best decision tree error of 243,000__. There are parameters which allow you to change the performance of the Random Forest much as we changed the maximum depth of the single decision tree. But one of the best features of Random Forest models is that they generally work reasonably even without this tuning.

## Missing Value Imputation

We just finished training our first machine learning models. To further improve the predictive power of the models we will have to __work on our dataset__.



We will start with handling missing values in the data. Most machine learning libraries (including scikit-learn) give an error if we try to build a model using data with missing values. So we'll need to choose a strategy to handle missing values.

We have already used a very simple strategy and dropped all rows containing missing values in the first example. To evaluate different approaches we will first load the full dataset and create a train-test split. (Note: As we cannot apply all imputation functions (e.g., mean) to categorical data we will only use numerical predictions in this simple example.)

In [20]:
# Load dataset
data = pd.read_csv(melbourne_file_path) 

# Target variable
y = data['Price']

# Drop non-numeric variables
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

### Simple Imputation using the Mean

One popular way to handle missing values is called imputation. Here, we fill in the missing values with some number. For instance, we can fill in the __mean__ value along each column. The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

In [21]:
from sklearn.impute import SimpleImputer

In [22]:
# Imputation
simple_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(simple_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(simple_imputer.transform(X_valid))

# "Repair" column names
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

To evaluate the performance of the approach, we modify our helper function (get_mae) to train and evaluate our model on different datasets:

In [23]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=1)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [24]:
mae_imputation = score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)
print("MAE using Imputation: {}".format(mae_imputation))

MAE using Imputation: 168341.16436233255


## Wrapping up

In this lecture, we learned how to build powerful machine learning models leveraging numerical variables. This included

* Data loading
* Variable Choice
* Data Splitting
* Model training
* Model Evaluation
* Hyperparameter Tuning
* Missing Value Imputation



## Further resources

* For a recap on machine learning models/algorithms, the basic resorce is [The Elements of Statistical Learning by Hastie et al (2008)](https://hastie.su.domains/ElemStatLearn/)
* Another, more hands-on resource is the [Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas
* To keep improving with your coding, view the [scikit-learn documentation](https://scikit-learn.org/stable/) and keep working on your own projects!

<img src="images/d3.png" style="width:50%; float:center;" />